In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import VarianceThreshold 
#feature의 variance가 적다는 말은 정보가 별로 없다는 말일 수도 있다. 100개의 샘플의 데이터가 99개 1 1개 0이면
#별로 배울게 없는 데이터이다.
from sklearn.feature_selection import SelectFromModel
#모델의 중요도에 따라서 선택하는 것.
#어떤 특성은 0으로 예측 했을 때 중요한 특성이 1로 하니까 중요도가 안 좋아지는 경우 ==> 중요한 특성
#그런데 0으로 하든 1로 하든 중요도가 변함이 없는 경우 ==> 안중요
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_rows', 100)

In [3]:
DEBUG = True

In [ ]:
if DEBUG :
    NROWS = 500000
else :
    NROWS = None

In [41]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')


In [6]:
train.shape

(595212, 59)

### 읽을 데이터가 너무 크다!
* pd.read_csv('....', nrows=integer) 또는
* sample의 fraction을 이용하는 것도 한 방법(random성 있음)
* imbalance data가 걱정 되는 경우 StratifiedSampling 이용


In [7]:
train = pd.read_csv('../input/train.csv')
train = train.sample(frac=0.1)
train.shape

(59521, 59)

### Stratified Sampling 깔끔하게 하기 꿀팁!

In [8]:
from sklearn.model_selection import StratifiedKFold

In [9]:
fold = StratifiedKFold(n_splits=10, random_state =42)

In [10]:
# index뽑기
for trn_idx, tst_idx in fold.split(train, train['target']):
    break

In [11]:
trn_idx

array([ 5948,  5949,  5950, ..., 59518, 59519, 59520])

In [12]:
train.iloc[trn_idx]

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
126091,315198,0,3,1,9,1,0,1,0,0,0,0,0,0,0,0,13,0,1,0,0.7,0.7,1.020417,11,0,-1,0,-1,6,1,1,2,1,104,2,0.447214,0.726363,0.441588,2.645751,0.0,0.9,0.3,2,1,8,0,7,4,13,8,2,1,7,1,0,1,0,0,0
270363,675514,0,3,1,6,1,0,0,1,0,0,0,0,0,0,0,4,1,0,0,0.9,0.7,0.727582,11,0,-1,0,-1,11,1,1,2,1,69,3,0.424264,1.118330,-1.000000,3.605551,0.9,0.8,0.8,3,3,5,2,8,3,8,8,2,4,7,0,1,1,1,1,0
36878,92393,0,2,2,0,0,6,0,0,1,0,0,0,0,0,0,8,1,0,0,0.8,1.0,0.865303,4,1,-1,0,1,11,1,0,0,1,36,2,0.374166,0.969206,0.379473,3.605551,0.4,0.1,0.1,3,5,7,5,9,1,8,3,2,1,6,0,0,0,1,0,0
240520,600919,0,0,2,1,1,0,1,0,0,0,0,0,0,0,0,11,1,0,0,0.3,0.0,0.848896,4,1,-1,0,1,1,1,1,0,1,65,1,0.316228,0.558888,0.350714,2.828427,0.4,0.7,0.4,2,1,7,5,10,4,2,4,1,5,10,0,1,0,1,0,0
150341,375765,0,3,1,3,0,0,0,1,0,0,0,0,0,0,0,2,0,0,1,0.7,1.1,1.488288,8,1,-1,8,0,13,1,1,0,1,4,2,0.565685,1.462606,0.518652,3.464102,0.8,0.6,0.3,4,1,8,5,11,3,8,1,1,3,8,1,0,0,0,0,0
469878,1174342,0,2,2,1,1,0,1,0,0,0,0,0,0,0,0,10,1,0,0,0.9,0.2,0.551135,7,1,-1,0,-1,11,1,1,2,1,103,3,0.316228,0.652190,0.358329,3.162278,0.5,0.1,0.0,1,2,8,2,9,3,10,7,2,1,7,0,0,1,0,0,0
46905,117585,0,7,1,7,1,0,0,1,0,0,0,0,0,0,0,13,1,0,0,0.9,0.2,0.648556,6,1,-1,0,1,0,1,1,0,1,78,3,0.400000,0.786245,0.384838,3.316625,0.9,0.5,0.8,2,2,8,4,9,3,8,7,0,1,8,0,0,1,0,0,0
551408,1378174,0,4,1,2,1,0,0,0,0,1,0,0,0,0,0,2,1,0,0,0.8,0.5,0.953284,11,1,1,0,0,11,1,1,2,1,67,2,0.374166,0.710793,0.300832,3.316625,0.6,0.0,0.8,2,2,8,4,12,2,10,4,0,4,8,0,0,0,1,0,0
165360,413426,0,1,2,2,1,0,0,0,0,1,0,0,0,0,0,10,0,1,0,0.9,0.4,0.828779,7,1,-1,0,-1,0,1,1,2,1,39,3,0.316070,0.628770,0.374433,2.449490,0.2,0.5,0.4,1,2,8,3,10,3,7,7,1,2,6,0,1,0,0,1,0
375352,938105,0,3,1,6,1,0,0,1,0,0,0,0,0,0,0,6,0,0,1,0.7,0.8,0.966954,11,1,1,0,0,1,1,1,2,1,25,3,0.374166,0.805495,-1.000000,3.605551,0.9,0.0,0.6,1,2,9,6,9,3,6,3,0,3,7,0,1,0,0,1,0


In [13]:
test.iloc[tst_idx]

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,0,1,8,1,0,0,1,0,0,0,0,0,0,0,12,1,0,0,0.5,0.3,0.610328,7,1,-1,0,-1,1,1,1,2,1,65,1,0.316228,0.669556,0.352136,3.464102,0.1,0.8,0.6,1,1,6,3,6,2,9,1,1,1,12,0,1,1,0,0,1
1,1,4,2,5,1,0,0,0,0,1,0,0,0,0,0,5,1,0,0,0.9,0.5,0.771362,4,1,-1,0,0,11,1,1,0,1,103,1,0.316228,0.606320,0.358329,2.828427,0.4,0.5,0.4,3,3,8,4,10,2,7,2,0,3,10,0,0,1,1,0,1
2,2,5,1,3,0,0,0,0,0,1,0,0,0,0,0,10,0,0,0,0.4,0.0,0.916174,11,1,-1,0,-1,14,1,1,2,1,29,3,0.400000,0.896239,0.398497,3.316625,0.6,0.6,0.6,2,3,7,4,6,3,12,4,0,2,4,0,0,0,0,0,0
3,3,0,1,6,0,0,1,0,0,0,0,0,0,0,0,4,1,0,0,0.1,0.2,-1.000000,7,1,-1,0,-1,1,1,1,2,1,40,2,0.374166,0.652110,0.381445,2.449490,0.1,0.5,0.5,2,1,7,3,12,1,13,5,1,0,5,1,0,1,0,0,0
4,4,5,1,7,0,0,0,0,0,1,0,0,0,0,0,4,1,0,0,0.9,0.4,0.817771,11,1,-1,0,-1,11,1,1,2,1,101,3,0.374166,0.812914,0.385097,3.316625,0.9,0.6,0.8,3,4,7,1,10,4,12,4,0,0,4,0,1,1,0,0,1
5,5,0,1,6,0,0,1,0,0,0,0,0,0,0,0,10,1,0,0,0.9,0.5,0.759934,9,1,-1,0,-1,11,0,0,2,1,11,2,0.316228,0.750922,0.368782,3.605551,0.7,0.9,0.4,2,1,9,5,9,4,12,8,1,4,9,1,0,1,0,1,0
6,6,0,1,3,0,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.1,0.1,-1.000000,6,1,-1,0,0,1,1,1,0,1,10,2,0.316228,0.724569,0.381445,3.316625,0.9,0.8,0.8,1,1,7,3,9,5,6,2,0,4,6,1,1,0,0,0,0
7,8,0,1,0,0,0,1,0,0,0,0,0,0,0,0,7,1,0,0,0.9,1.1,1.145917,7,1,-1,0,-1,11,1,1,2,1,103,3,0.316228,0.586610,0.358329,2.645751,0.8,0.9,0.5,2,2,8,4,11,2,8,3,1,4,9,0,1,0,0,0,0
8,10,0,1,7,0,0,0,1,0,0,0,0,0,0,0,6,1,0,0,0.4,0.0,0.597390,11,0,1,1,0,2,-1,0,0,1,104,2,0.447214,1.640147,0.367423,3.741657,0.9,0.3,0.0,2,2,9,7,9,0,10,5,1,4,6,0,0,1,0,0,0
9,11,1,1,6,0,0,0,0,0,1,0,0,0,0,0,7,0,1,0,0.9,1.0,-1.000000,11,0,-1,0,-1,4,1,1,2,1,104,2,0.447214,1.041336,0.422493,3.464102,0.0,0.9,0.7,2,1,7,1,9,1,11,6,1,6,10,0,1,1,0,0,0


깔끔!

In [14]:
cat_cols = [col for col in train.columns if 'cat' in col]

In [15]:
cat_cols

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [16]:
train[cat_cols[0]].value_counts()

 1    43221
 2    12420
 3     2768
 4     1088
-1       24
Name: ps_ind_02_cat, dtype: int64

In [17]:
# category의 유니크한 개수
for col in cat_cols:
    print(col,':', train[col].nunique() )
    #high cardinality : ps_cal_11_cat ==> 104개 
    #(one-hot-encoding하게 되면 머리아파짐,, 다른 encoding방법이 필요)

ps_ind_02_cat : 5
ps_ind_04_cat : 3
ps_ind_05_cat : 8
ps_car_01_cat : 13
ps_car_02_cat : 2
ps_car_03_cat : 3
ps_car_04_cat : 10
ps_car_05_cat : 3
ps_car_06_cat : 18
ps_car_07_cat : 3
ps_car_08_cat : 2
ps_car_09_cat : 6
ps_car_10_cat : 3
ps_car_11_cat : 104


In [18]:
train.shape

(59521, 59)

In [19]:
train = train.drop_duplicates()
train.shape
#중복된 값 없음

(59521, 59)

In [20]:
test = test.drop_duplicates()
test.shape

(892816, 58)

dummy feature에 one_hot_encoding을 실시할 것

In [21]:
train.info() #object는 없다 --> label_encoding 필요 없음

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59521 entries, 216324 to 440740
Data columns (total 59 columns):
id                59521 non-null int64
target            59521 non-null int64
ps_ind_01         59521 non-null int64
ps_ind_02_cat     59521 non-null int64
ps_ind_03         59521 non-null int64
ps_ind_04_cat     59521 non-null int64
ps_ind_05_cat     59521 non-null int64
ps_ind_06_bin     59521 non-null int64
ps_ind_07_bin     59521 non-null int64
ps_ind_08_bin     59521 non-null int64
ps_ind_09_bin     59521 non-null int64
ps_ind_10_bin     59521 non-null int64
ps_ind_11_bin     59521 non-null int64
ps_ind_12_bin     59521 non-null int64
ps_ind_13_bin     59521 non-null int64
ps_ind_14         59521 non-null int64
ps_ind_15         59521 non-null int64
ps_ind_16_bin     59521 non-null int64
ps_ind_17_bin     59521 non-null int64
ps_ind_18_bin     59521 non-null int64
ps_reg_01         59521 non-null float64
ps_reg_02         59521 non-null float64
ps_reg_03         59521

### Metadata
To facilitate the data management, we'll store meta-inforamtation about the variablel in a DataFrame. This will be helpful when we wnat to select specific variables for analysis, visualization, modeling,...

Concretely we will store:
* role : input, ID, target
* level : nominal, interval, ordinal, binary
* keep : True, False
* dtype : int, float, str


In [42]:
data = []
for f in train.columns:
    if f == 'target' :
         role = 'target'
    elif f == 'id':
        role = 'id'
    else : 
        role = 'input'
        
    #Defining the level
    if 'bin' in f or f == 'target':
        level =  'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif train[f].dtype == float:
        level = 'interval'
    elif train[f].dtype == int:
        level = 'ordinal'
        
    #Initialize keep to True for all variables except for id
    keep = True
    if f == 'id':
        keep = False
    #Defining the data type    
    dtype = train[f].dtype
    #Crearting a Dict that contains all the metadata for the variable
    f_dict = {
        'varname':f,
        'role':role,
        'level':level,
        'keep': keep,
        'dtype':dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns = ['varname','role','level','keep','dtype'])
meta.set_index('varname',inplace=True)

In [43]:
meta

,role,level,keep,dtype
varname,,,,
id,id,nominal,False,int64
target,target,binary,True,int64
ps_ind_01,input,ordinal,True,int64
ps_ind_02_cat,input,nominal,True,int64
ps_ind_03,input,ordinal,True,int64
ps_ind_04_cat,input,nominal,True,int64
ps_ind_05_cat,input,nominal,True,int64
ps_ind_06_bin,input,binary,True,int64
ps_ind_07_bin,input,binary,True,int64


Example to extract all nominal variables that are not dropped

In [44]:
meta[(meta.level == 'nominal') & (meta.keep)].index

Index(['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat',
       'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat',
       'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat',
       'ps_car_10_cat', 'ps_car_11_cat'],
      dtype='object', name='varname')

Below the number of variables per role and level are displayed

In [45]:
pd.DataFrame({'count': meta.groupby(['role','level'])
             ['role'].size()}).reset_index()

,role,level,count
0,id,nominal,1
1,input,binary,17
2,input,interval,10
3,input,nominal,14
4,input,ordinal,16
5,target,binary,1


정리를 이쁘게 했다.

### Descriptive Statistics
We can also apply the describe method on the dataframe. However, it doesn't make much sense to calculate the mean, std,... on categorical variables and the id variable. We'll explore the categorical variables visually later.

Thanks to our meta file we can easily select the variables on which we want to compute the descriptive statistics. To keep things clear, we'll do this per data type.

Interval variables

In [46]:
categorical_feat = [col for col in train.columns if 'cat' in col]

In [47]:
categorical_feat #이런식으로 뽑는 것도 가능하다

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [48]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
train[v].describe()

,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03
count,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,0.610991,0.439184,0.551102,0.379945,0.813265,0.276256,3.065899,0.449756,0.449589,0.449849
std,0.287643,0.404264,0.793506,0.058327,0.224588,0.357154,0.731366,0.287198,0.286893,0.287153
min,0.000000,0.000000,-1.000000,-1.000000,0.250619,-1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,0.200000,0.525000,0.316228,0.670867,0.333167,2.828427,0.200000,0.200000,0.200000
50%,0.700000,0.300000,0.720677,0.374166,0.765811,0.368782,3.316625,0.500000,0.400000,0.500000
75%,0.900000,0.600000,1.000000,0.400000,0.906190,0.396485,3.605551,0.700000,0.700000,0.700000
max,0.900000,1.800000,4.037945,1.264911,3.720626,0.636396,3.741657,0.900000,0.900000,0.900000


reg variables
* only ps_reg_03 has missing values
* the range(min to max) differs between the variables. We could apply scaling(e.g StandardScaler), but it depends on the classifier we will want to use.
* linear regression일 경우, feature들 간의 distribution이 너무 다르, 특정 var에 쏠리는 경우가 발생함. StandardScaler사용.(선형모델)
* Tree 모델을 사용할 경우, 절대값의 크기에 영향 별로 안받음. 굳이 안해도 됨

car variables
* ps_car_12 and ps_car_15 have missing values
* again, the range differs and we could apply scaling.

calc variables
* no missing values
* this seems to be some kind of ratio as the maximum is 0.9
* all three `_calc` variables have very similar distributions

__Overall,__ we can see that the range of the interval variables is rather small. Perhaps some transformation(e.g log) is already applied in order to anoymize the data?

Ordinal variables





In [49]:
v = meta[(meta.level == 'ordinal') & (meta.keep)].index
train[v].describe()

,ps_ind_01,ps_ind_03,ps_ind_14,ps_ind_15,ps_car_11,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14
count,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,1.900378,4.423318,0.012451,7.299922,2.346072,2.372081,1.885886,7.689445,3.005823,9.225904,2.339034,8.433590,5.441382,1.441918,2.872288,7.539026
std,1.983789,2.699902,0.127545,3.546042,0.832548,1.117219,1.134927,1.334312,1.414564,1.459672,1.246949,2.904597,2.332871,1.202963,1.694887,2.746652
min,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,5.000000,2.000000,2.000000,1.000000,7.000000,2.000000,8.000000,1.000000,6.000000,4.000000,1.000000,2.000000,6.000000
50%,1.000000,4.000000,0.000000,7.000000,3.000000,2.000000,2.000000,8.000000,3.000000,9.000000,2.000000,8.000000,5.000000,1.000000,3.000000,7.000000
75%,3.000000,6.000000,0.000000,10.000000,3.000000,3.000000,3.000000,9.000000,4.000000,10.000000,3.000000,10.000000,7.000000,2.000000,4.000000,9.000000
max,7.000000,11.000000,4.000000,13.000000,3.000000,5.000000,6.000000,10.000000,9.000000,12.000000,7.000000,25.000000,19.000000,10.000000,13.000000,23.000000


* Only one missing variable: ps_car_11
* We could apply sacling to deal with the different ranges

Binary variables

In [51]:
v = meta[(meta.level == 'binary') & (meta.keep)].index
train[v].describe()

,target,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,0.036448,0.393742,0.257033,0.163921,0.185304,0.000373,0.001692,0.009439,0.000948,0.660823,0.121081,0.153446,0.122427,0.627840,0.554182,0.287182,0.349024,0.153318
std,0.187401,0.488579,0.436998,0.370205,0.388544,0.019309,0.041097,0.096693,0.030768,0.473430,0.326222,0.360417,0.327779,0.483381,0.497056,0.452447,0.476662,0.360295
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


- A priori in the train data is 3.645%, which is __strongly imbalanced.__
- From the means we can conclude that for most variables the value is zero in most cases. (대부분이 보험 청구가 안되었다.)

### Handling imbalanced classes
As we mentioned above the proportion of records with target = 1 is far less than target=0. This lead to a model that has great accuracy but does have any added value in practice. Two possible strategies to deal with this problem are:
- oversampling records with target=1
- undersampling records with target=0
There are many more strategies of course and MachineLearningMastery.com gives a nice overview. As we have a rather large training set, se can go for __undersamplling.__

얼마나 1을 잘 맞추느냐가 중요한 상황. gini나 ROC AUC를 사용하는 이유.  
undersampling 많은 걸 줄임, target이 0인 걸 줄일 것


SMOTE!

In [55]:
desired_apriori = 0.10

#Get the indices per target value
idx_0 = train[train.target ==0].index
idx_1 = train[train.target == 1].index

#Get original number of records per target value
nb_0 = len(train.loc[idx_0])
nb_1 = len(train.loc[idx_1])

# Calculate the undersampling rate and resulting number of records with target =0
undersampling_rate = ((1-desired_apriori)*nb_1) / (nb_0*desired_apriori)
undersampled_nb_0 = int(undersampling_rate*nb_0)
print('Rate to undersample records with target=0: {}'.format(undersampling_rate))
print('Number of records with target=0 after undersampling: {}'.format(undersampled_nb_0))

#Randomly select records with target=0 to get at the desired a priori
undersampled_idx = shuffle(idx_0, random_state=37, n_samples = undersampled_nb_0)

#construct list with remaining indices
idx_list = list(undersampled_idx) + list(idx_1)

#return undersampled data frame
train = train.loc[idx_list].reset_index(drop=True) #shuffle해줬으니까 순서대로

Rate to undersample records with target=0: 0.34043569687437886
Number of records with target=0 after undersampling: 195246


In [56]:
nb_0

573518

In [57]:
nb_1

21694

### Data Quality Checks

* Checking missing values  
Missings are represeted as -1

In [58]:
vars_with_missing = []
for f in train.columns :
    missings = train[train[f] == -1][f].count()
    if missings > 0 :
        vars_with_missing.append(f)
        missings_perc = missings/train.shape[0]
        
        print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, missings_perc))
        
print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))


Variable ps_ind_02_cat has 103 records (0.05%) with missing values
Variable ps_ind_04_cat has 51 records (0.02%) with missing values
Variable ps_ind_05_cat has 2256 records (1.04%) with missing values
Variable ps_reg_03 has 38580 records (17.78%) with missing values
Variable ps_car_01_cat has 62 records (0.03%) with missing values
Variable ps_car_02_cat has 2 records (0.00%) with missing values
Variable ps_car_03_cat has 148367 records (68.39%) with missing values
Variable ps_car_05_cat has 96026 records (44.26%) with missing values
Variable ps_car_07_cat has 4431 records (2.04%) with missing values
Variable ps_car_09_cat has 230 records (0.11%) with missing values
Variable ps_car_11 has 1 records (0.00%) with missing values
Variable ps_car_14 has 15726 records (7.25%) with missing values
In total, there are 12 variables with missing values


- ps_car_03_cat and ps_car_05_cat have a large proportion of recoreds with missing values. Remove these variables.
- For the other categorical variables with missing values, we can leave the missing value -1 as such.
- ps_reg_03(continuois) has missing values for 18% of all records. Replace by the mean
- ps_car_11(ordinal) has only 5 records with missing values. Replace by the mode.
- ps_car_12(continuous)has only 1records with missing value. Replace by the mean.
- ps_car_14(continuous) has missing values for 7% of all records. Replace by the mean.

### Missing Value는 함부로 채우면 안된다
모델을 학습시켜서 채우는 방법도 있다.

가끔씩 missing value값이 확 뛸 때가 있음. 이경우는 missing value에 정보가 있다는 말일 수 도 있음. 하지만 porto에서는 정보가 없다.

In [59]:
#Dropping the variables with too many missing values
vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
train.drop(vars_to_drop, inplace=True, axis=1)
meta.loc[(vars_to_drop), 'keep'] = False

In [99]:
mean_imp.fit_transform(train[['ps_car_14']]).shape

(216940, 1)

In [98]:
mean_imp.fit_transform(train[['ps_car_14']]).ravel().shape

(216940,)

In [60]:
#Imputing with the mean or mode
mean_imp = Imputer(missing_values=-1, strategy='mean', axis=0)
mode_imp = Imputer(missing_values=-1, strategy='most_frequent', axis=0)
train['ps_reg_03'] = mean_imp.fit_transform(train[['ps_reg_03']]).ravel()
train['ps_reg_car_12'] = mean_imp.fit_transform(train[['ps_car_12']]).ravel()
train['ps_car_14'] = mean_imp.fit_transform(train[['ps_car_14']]).ravel()
train['ps_car_11'] = mean_imp.fit_transform(train[['ps_car_11']]).ravel()

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


### Checing the cardinality of the categorical variables
Cardinality refers to the number of different values in a variable. As we will create dummy variables from the categorical variables later on, we need to check whether there are variables with many distinct values. We should handle these variables differently as they would result in many dummy variables.

In [89]:
v = meta[(meta.level =='nominal') & (meta.keep)].index
for f in v:
    dist_values = train[f].value_counts().shape[0]
    print('Varaible {} has {} distinct values'.format(f, dist_values))

Varaible ps_ind_02_cat has 5 distinct values
Varaible ps_ind_04_cat has 3 distinct values
Varaible ps_ind_05_cat has 8 distinct values
Varaible ps_car_01_cat has 13 distinct values
Varaible ps_car_02_cat has 3 distinct values
Varaible ps_car_04_cat has 10 distinct values
Varaible ps_car_06_cat has 18 distinct values
Varaible ps_car_07_cat has 3 distinct values
Varaible ps_car_08_cat has 2 distinct values
Varaible ps_car_09_cat has 6 distinct values
Varaible ps_car_10_cat has 3 distinct values
Varaible ps_car_11_cat has 104 distinct values


In [94]:
train[v[0]].value_counts().shape[0]

5

In [100]:
train[v[0]].nunique() # 같은 결과

5

One hot Encoding 하는데 Cardinality가 높다면, 많은 distinct values들이 있다면, 1000개의 칼럼 계산할게 너무 많아짐.

Only ps_car_11 has many distinct values, although it is still reasonable.

EDIT: nickycan made an excellent remark on the fact that my first solution could lead to data leakage. He also pointed me to another kernel made by oliver which deals with that. I therefore replaced this part with the kernel of oliver. All credits go to him. It is so great what you can learn by participating in the Kaggle competitions :)

74개의 칼럼이 생기게 되는데...Accept할지 말지는 알아서 결정.

칼럼이 너무 많아지면, 트리모델을 사용할 경우, 트리의 깊이도 깊어지고, 중요한 질문을 못 건드릴 가능성 높아짐, 답으로 가기 위한 과정에서 멀어짐, 원하지 않는 split이 생길 수 있음. one hot encoding많아지면 힘들다... ==> mean encoding(Overfitting 가능성) OR Frequency Encoding 사용한다.